In [132]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import pandas as pd
import seaborn as sns
import pathlib as Path
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, train_test_split, ShuffleSplit

In [133]:
df = pd.read_csv('../input/train.csv')

In [134]:
def split_datetime(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df['year_' + column_name] = df[column_name].dt.year
    df['month_' + column_name] = df[column_name].dt.month
    df['day_' + column_name] = df[column_name].dt.day
    df['weekday_' + column_name] = df[column_name].dt.weekday + 1
    df['hour_' + column_name] = df[column_name].dt.hour
    df['minute_' + column_name] = df[column_name].dt.minute
    return df

In [135]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


# PREPROCESSING

### Splitting a datetime

In [136]:
new_df = split_datetime(df, 'pickup_datetime')
new_df.shape

(1458644, 17)

### Column 'store_and_fwd_flag' into binary values

In [137]:
# new_df['store_and_fwd_flag'] = pd.get_dummies(new_df['store_and_fwd_flag'])
# new_df.head()

### Applying some filters

#### 1st filter

In [138]:
new_df = new_df[new_df['passenger_count'] != 0]
new_df.shape

(1458584, 17)

#### 2nd filter

In [139]:
new_df = new_df[new_df['trip_duration'] <= 1800]
new_df.shape

(1345471, 17)

In [140]:
new_df = new_df[new_df['trip_duration'] >= 360]
new_df.shape

(1037034, 17)

### Selecting columns to use

In [141]:
selected_columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
                    'dropoff_latitude', 'month_pickup_datetime', 'hour_pickup_datetime']

## Defining target & features

In [142]:
X_full = new_df[selected_columns]
y_full = new_df['trip_duration']
X_full.shape, y_full.shape

((1037034, 6), (1037034,))

### Splitting my dataset

In [143]:
X_train, X_valid, y_train, y_valid = train_test_split(
            X_full, y_full, test_size=0.33, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((694812, 6), (342222, 6), (694812,), (342222,))

In [144]:
#X_train, X_valid, y_train, y_valid = train_test_split(
#            X_train_used, y_train_used, test_size=0.33, random_state=50)
#X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

# Training model

In [145]:
#rf = RandomForestRegressor()

In [146]:
xbr = XGBRegressor(n_jobs=-1)

In [147]:
#params_grid = {
#    'colsample_bytree':[0.4,0.6,0.8],
#    'gamma':[0.03,0.3],
#    'min_child_weight':[1.5,6],
#    'learning_rate':[0.1,0.07],
#    'reg_alpha':[1e-5, 1e-2,  0.75],
#    'reg_lambda':[1e-5, 1e-2, 0.45],
#    'subsample':[0.6,0.95],
#    'max_depth': [3, 5],
    #'min_samples_leaf': [1, 3, 8, 12]
#}

In [148]:
# kf = KFold(n_splits=5, random_state=1) 

In [149]:
# gsc = GridSearchCV(xbr, params_grid, n_jobs=-1, verbose=10, scoring='neg_mean_squared_log_error')

In [150]:
# gsc.fit(X_train, y_train)

In [151]:
# gsc.estimator

In [152]:
# gsc.best_index_

In [153]:
# gsc.best_score_

In [154]:
# gsc.best_params_

In [155]:
# gsc.n_splits_

In [156]:
#cv = ShuffleSplit(n_splits=5, train_size=0.75, random_state=0)

In [157]:
# rf_v2 = RandomForestRegressor(max_depth=15, min_samples_leaf=12)

In [158]:
xbr.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [159]:
xbr.feature_importances_

array([0.27473503, 0.19699925, 0.23714195, 0.17835715, 0.04107626,
       0.07169034], dtype=float32)

In [161]:
#losses = -cross_val_score(rf_v2, X_train_used, y_train_used, cv=gsc.best_index_, scoring='neg_mean_squared_log_error')
#losses.mean()

Real score value

In [162]:
#losses = [np.sqrt(l) for l in losses]
#np.mean(losses)

In [163]:
#rf_v2.fit(X_train_used, y_train_used)

In [164]:
#rf_v2.feature_importances_

In [165]:
y_pred_valid = xbr.predict(X_valid)

In [166]:
#y_pred = rf_v2.predict(X_train_unused)

In [167]:
y_pred_valid.mean()

832.05096

In [168]:
np.mean(y_valid)

833.1165179328038

In [169]:
df_test = pd.read_csv('../input/test.csv')

In [170]:
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [171]:
df_test = split_datetime(df_test, 'pickup_datetime')

In [172]:
# df_test['store_and_fwd_flag'] = pd.get_dummies(df_test['store_and_fwd_flag'])

In [173]:
X_test = df_test[selected_columns]

In [174]:
y_pred_test = xbr.predict(X_test)

In [175]:
y_pred_test.mean()

847.7221

In [176]:
submission = pd.read_csv('../input/sample_submission.csv') 
submission.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [177]:
submission['trip_duration'] = y_pred_test

In [178]:
submission.describe()

,trip_duration
count,625134.000000
mean,847.602905
std,244.326309
min,461.763947
25%,716.721725
50%,773.404114
75%,861.013885
max,2569.468750


In [179]:
submission.to_csv('submission.csv', index=False)

In [180]:
!ls

__notebook_source__.ipynb  submission.csv
